# Lab 2: Messages - O Sistema Circulatório dos Agentes

<img src="./assets/LC_Messages.png" width="500">

## 🎯 O que você vai aprender neste Lab

**Messages** são a unidade fundamental de contexto para modelos no LangChain. Elas representam os inputs e outputs dos modelos, carregando tanto o conteúdo quanto os metadados necessários para representar o estado de uma conversa.

### Por que Messages são importantes?

1. **Padronização**: Messages são padronizadas entre provedores (OpenAI, Anthropic, Databricks)
2. **Flexibilidade**: Você pode trocar de modelo sem mudar o código
3. **Contexto Rico**: Carregam metadados como tokens usados, modelo, etc.
4. **Histórico Completo**: Mantêm todo o histórico da interação

## 📬 Tipos de Messages no LangChain

O LangChain define vários tipos de mensagens que seguem de perto os tipos/roles usados pelos provedores de LLM:

| Tipo | Role | Descrição | Exemplo |
|------|------|-----------|---------|
| **SystemMessage** | `system` | Define o papel/comportamento do agente | "Você é um comediante" |
| **HumanMessage** | `user` | Mensagem do usuário | "Conte uma piada" |
| **AIMessage** | `assistant` | Resposta do modelo | "Por que o programador..." |
| **ToolMessage** | `tool` | Resultado de uma ferramenta | "Resultado: 42" |

### Fluxo típico de mensagens:

```
┌─────────────────────────────────────────────────────────────┐
│                    FLUXO DE MENSAGENS                        │
│                                                              │
│  SystemMessage ──→ Define persona ("Você é um analista")    │
│        ↓                                                     │
│  HumanMessage  ──→ Pergunta do usuário ("Qual é o total?")  │
│        ↓                                                     │
│  AIMessage     ──→ Agente raciocina e chama tool            │
│        ↓                                                     │
│  ToolMessage   ──→ Ferramenta retorna resultado             │
│        ↓                                                     │
│  AIMessage     ──→ Resposta final formatada                 │
└─────────────────────────────────────────────────────────────┘
```

**Importante**: As mensagens não são passadas de nó para nó, mas armazenadas em um "scratchpad" persistente compartilhado por todos os nós do agente.

## ⚙️ Setup - Configuração do Ambiente

Instalação das bibliotecas e configuração do MLflow para rastreamento automático.

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

## 👨‍💻 HumanMessage e 🤖 AIMessage

Vamos começar com o par mais básico de mensagens:
- **HumanMessage**: Representa uma mensagem do usuário
- **AIMessage**: Representa a resposta do modelo

### Criando um agente simples

Neste exemplo, criamos um agente "comediante full-stack" e enviamos uma `HumanMessage` explícita.

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(
    model=llm, 
    system_prompt="You are a full-stack comedian"
)

In [0]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

### Enviando uma HumanMessage

Criamos explicitamente uma `HumanMessage` e invocamos o agente. O resultado contém uma lista de todas as mensagens da conversa.

### Acessando o conteúdo da resposta

A resposta do agente está em `result["messages"]`. O último elemento (`[-1]`) é sempre a resposta final do modelo.

In [0]:
print(result["messages"][-1].content)

In [0]:
print(type(result["messages"][-1]))

In [0]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

### Iterando por todas as mensagens

Podemos ver o tipo e conteúdo de cada mensagem no histórico:

## 📝 Formatos Alternativos de Mensagens

O LangChain oferece várias formas de representar mensagens, desde objetos explícitos até formatos simplificados.

### Strings Simples

Em situações onde o LangChain pode inferir o role do contexto, uma string simples é suficiente:
- `system_prompt="..."` → Convertido para SystemMessage
- `{"messages": "..."}` → Convertido para HumanMessage

In [ ]:
agent = create_agent(
    model=llm,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [0]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

### Dicionários

Você pode usar dicionários com `role` e `content` explícitos. Isso é útil quando você quer ter controle total sobre o tipo da mensagem.

In [0]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

### Roles Disponíveis

Os roles mapeiam para os tipos de mensagem:

| Role | Tipo LangChain | Uso |
|------|----------------|-----|
| `system` | SystemMessage | Define comportamento do agente |
| `user` | HumanMessage | Input do usuário |
| `assistant` | AIMessage | Resposta do modelo |
| `tool` | ToolMessage | Resultado de ferramenta |

```python
messages = [
    {"role": "system", "content": "Você é um expert em haikus"},
    {"role": "user", "content": "Escreva um haiku"},
    {"role": "assistant", "content": "Folhas ao vento..."}  # Contexto prévio
]
```

## 🔧 ToolMessage - Vendo Ferramentas em Ação

Quando um agente usa uma ferramenta, o resultado retorna como uma `ToolMessage`. Vamos criar uma ferramenta para verificar haikus e observar todas as mensagens geradas.

### A ferramenta `check_haiku_lines`

Esta ferramenta valida se um haiku tem exatamente 3 linhas (característica fundamental de um haiku).

In [0]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

### Criando o agente poeta com verificação

O system prompt instrui o agente a sempre verificar seu trabalho usando a ferramenta.

In [ ]:
agent = create_agent(
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [0]:
result = agent.invoke({"messages": "Please write me a poem"})

### Analisando as mensagens geradas

Observe que agora temos 4 mensagens no fluxo:
1. **HumanMessage**: "Please write me a poem"
2. **AIMessage** com tool_call: O modelo chama `check_haiku_lines`
3. **ToolMessage**: Resultado da verificação
4. **AIMessage**: Resposta final com o haiku

In [0]:
result["messages"][-1].content

In [0]:
print(len(result["messages"]))

In [0]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

### Usando `pretty_print()` para visualização

O método `pretty_print()` formata as mensagens de forma legível, mostrando claramente o tipo e conteúdo de cada uma.

## 📊 Metadados das Mensagens

As mensagens contêm muito mais do que apenas o conteúdo. Vamos explorar os metadados disponíveis que são úteis para debugging, monitoramento e otimização.

In [0]:
result

### Acessando a última mensagem completa

Ao selecionar uma mensagem específica, você pode ver todos os seus atributos.

In [0]:
result["messages"][-1]

### `usage_metadata` - Uso de Tokens

Informações sobre consumo de tokens - essencial para controle de custos:
- `input_tokens`: Tokens de entrada (prompt)
- `output_tokens`: Tokens de saída (resposta)
- Detalhes de cache quando disponível

In [0]:
result["messages"][-1].usage_metadata

### `response_metadata` - Informações do Provedor

Detalhes específicos do provedor de LLM:
- Nome do modelo
- Configurações usadas
- Informações de latência

In [0]:
result["messages"][-1].response_metadata

## 🎮 Sua Vez - Experimente!

Mude o system prompt, use `pretty_print()` para visualizar mensagens, ou explore o objeto `result` por conta própria.

**O que observar:**
- Tipos de mensagem (Human, AI, Tool)
- Metadados associados
- O histórico completo fornece uma trilha de auditoria da atividade do agente!

In [ ]:
agent = create_agent(
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [0]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

## 📚 Resumo e Próximos Passos

### O que aprendemos:
- **Messages** são a unidade fundamental de comunicação no LangChain
- Existem 4 tipos principais: **System**, **Human**, **AI**, e **Tool**
- Messages podem ser criadas como objetos, strings, ou dicionários
- Cada mensagem carrega metadados úteis (tokens, modelo, etc.)
- O resultado final contém todo o histórico da interação

### Próximo Lab:
No **Lab 3 - Streaming**, você aprenderá a receber respostas em tempo real, token por token, para criar experiências mais responsivas!